In [1]:
from __future__ import print_function
%matplotlib inline
%pylab inline
# %load_ext memory_profiler
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy
import itertools
from tabulate import tabulate
from pomegranate import *
from pomegranate import BayesianNetwork
import iac_org as iacO

Populating the interactive namespace from numpy and matplotlib


In [2]:
attributes = ["gang", "age", "edu", "mar", "occ", "name"]

def prob(evidences, atts):
    count, total = 0, 0
    for datapoint in iacO.data:
        inc = True
        for key in evidences:
            inc = inc and datapoint[key] == evidences[key]
        if inc:
            total += 1
            f = True
            for key in atts:
                f = f and datapoint[key] == atts[key]
            if f:
                count += 1
    return count, total, float(count)/total

def enums(key):
    vals = set()
    for datapoint in iacO.data:
        if datapoint[key] not in vals:
            vals.add(datapoint[key])
    return list(vals)

In [3]:
attributes = ["gang", "age", "edu", "mar", "occ", "name"]
distributions = {}
for att in attributes:
    probs = {}
    vals = enums(att)
    for val in vals:
        probs[val] = prob({},{att:val})[2]
    distributions[att] = DiscreteDistribution(probs)

dataInArray = []
n = {}
for datapoint in iacO.data:
    row = []
    for att in attributes:
        row.append(datapoint[att])
    dataInArray.append(row)
print(len(dataInArray))
# print(tabulate(dataInArray))

27


In [4]:
src = []
dist = []
states = []
for att in attributes:
    src.append(enums(att))
    dist.append(distributions[att])
CPT = []
cpt_dict = {}
c = 0
for element in itertools.product(*src):
    temp = list(element)
    if temp in dataInArray:
        CPT.append(temp+[1])
        c += 1
        cpt_dict[element] = 1
    else:
        CPT.append(temp+[0])
        cpt_dict[element] = 0
# CPT
# print(cpt_dict)
# print(tabulate(CPT, headers=attributes+["prob"]))

In [5]:
cond = ConditionalProbabilityTable(CPT, dist[:-1])
name = State(cond, name="name")
gang = State(distributions["gang"], name="gang")
age = State(distributions["age"], name="age")
edu = State(distributions["edu"], name="edu")
mar = State(distributions["mar"], name="mar")
occ = State(distributions["occ"], name="occ")

model = BayesianNetwork("Jets and Sharks")
model.add_states(gang, age, edu, mar, occ, name)

model.add_edge(gang, name)
model.add_edge(age, name)
model.add_edge(edu, name)
model.add_edge(mar, name)
model.add_edge(occ, name)
model.bake()
# print(model.predict_proba({}))

In [7]:
# print(prob({'gang': 'Jets', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'occ': 'bookie' ,'mar': 'single'}))
# print(prob({'name': 'Lance', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'name': 'Lance', 'occ': 'pusher'}))
# print(model.predict_proba({'gang': 'Sharks', 'age': '20s'}))
# print(model.predict_proba({'gang': 'Jets'}))
print(model.predict_proba({'name': 'John'}))
# print(prob({},{'gang': 'Jets'}))
# print(prob({},{'name': 'John'x}))

[{
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "Jets" :1.0,
            "Sharks" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "20s" :1.0,
            "30s" :0.0,
            "40s" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "HS" :0.0,
            "JH" :1.0,
            "COL" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "single" :0.0,
            "married" :1.0,
            "divorced" :0.0
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
     